In [59]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Lambda, Input, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob

In [60]:
import requests
import zipfile
import os
import kagglehub

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
filename = "diabetes.csv"

# Download
r = requests.get(url)
open(filename, 'wb').write(r.content)

path = kagglehub.dataset_download("uraninjo/augmented-alzheimer-mri-dataset")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/augmented-alzheimer-mri-dataset


In [61]:
from sklearn.model_selection import train_test_split

# Load the dataset
diabetes_df = pd.read_csv('diabetes.csv', header=None)

# Split the data into features (X) and target (y)
X = diabetes_df.iloc[:, :-1]
y = diabetes_df.iloc[:, -1]

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Concatenate features and target for saving
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Save the training and testing data to new CSV files
train_df.to_csv('diabetes_train.csv', index=False, header=False)
test_df.to_csv('diabetes_test.csv', index=False, header=False)

print("Training data saved to diabetes_train.csv")
print("Testing data saved to diabetes_test.csv")

Training data saved to diabetes_train.csv
Testing data saved to diabetes_test.csv


In [62]:
image_size = [224, 224]
Inception = InceptionV3(input_shape=image_size + [3], weights='imagenet', include_top=False)

In [63]:
for layer in Inception.layers:
  layer.trainable = False

In [64]:
train_path = "/content/diabetes_train.csv"
test_path = "/content/diabetes_test.csv"

In [65]:
folder = glob('/content/diabetes_train.csv')

In [66]:
x = Flatten()(Inception.output)


In [67]:
prediction = Dense(len(folder), activation='softmax')(x)
model = Model(inputs=Inception.input, outputs=prediction)
model.summary()

Model: "functional_122"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 111, 111,  │        864 │ input_layer_26[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 109, 109,  │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 109, 109,  │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 54, 54,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 54, 54,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 52, 52,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 21,853,985 (83.37 MB)

 Trainable params: 51,201 (200.00 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [68]:
model.compile(
    loss= 'categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

In [69]:
print("Missing values per column:")
print(diabetes_df.isnull().sum())

print("\nDescriptive statistics:")
display(diabetes_df.describe())

Missing values per column:
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

Descriptive statistics:


,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [70]:
from sklearn.preprocessing import StandardScaler

# Identify columns with potential outliers (0 values that are not plausible)
# Based on domain knowledge for diabetes, 0 glucose, blood pressure, skin thickness, insulin, and BMI are not realistic.
cols_with_outliers_as_zero = [1, 2, 3, 4, 5]

# Replace 0 values in these columns with NaN
for col in cols_with_outliers_as_zero:
    diabetes_df[col] = diabetes_df[col].replace(0, np.nan)

# Impute missing values with the mean of each column
diabetes_df.fillna(diabetes_df.mean(), inplace=True)

# Separate features (X) and target (y) again after imputation
X = diabetes_df.iloc[:, :-1]
y = diabetes_df.iloc[:, -1]

# Apply StandardScaler to the feature columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert the scaled features back to a DataFrame for potential future use
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Display the first few rows of the scaled features and descriptive statistics to verify
display(X_scaled_df.head())
display(X_scaled_df.describe())

,0,1,2,3,4,5,6,7
0,0.639947,0.865108,-0.033518,6.655021e-01,-3.345079e-16,0.166292,0.468492,1.425995
1,-0.844885,-1.206162,-0.529859,-1.746338e-02,-3.345079e-16,-0.852531,-0.365061,-0.190672
2,1.233880,2.015813,-0.695306,8.087936e-16,-3.345079e-16,-1.332833,0.604397,-0.105584
3,-0.844885,-1.074652,-0.529859,-7.004289e-01,-7.243887e-01,-0.634212,-0.920763,-1.041549
4,-1.141852,0.503458,-2.680669,6.655021e-01,1.465506e-01,1.548980,5.484909,-0.020496


,0,1,2,3,4,5,6,7
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-6.476301e-17,-3.561966e-16,6.915764e-16,7.956598e-16,-3.330669e-16,3.515706e-16,2.451743e-16,1.931325e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-2.554131e+00,-4.004245e+00,-2.521670e+00,-1.665945e+00,-2.075119e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-7.212214e-01,-6.953060e-01,-4.727737e-01,-4.007289e-01,-7.215397e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.540881e-01,-1.675912e-02,8.087936e-16,-3.345079e-16,-8.363615e-03,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.103090e-01,6.282695e-01,3.240194e-01,-3.345079e-16,6.029301e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.541850e+00,4.102655e+00,7.950467e+00,8.126238e+00,5.042087e+00,5.883565e+00,4.063716e+00


In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define a new Sequential model
model = Sequential()

# Add Dense layers
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_102 (Dense)               │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,521 (45.00 KB)

 Trainable params: 11,521 (45.00 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
from tensorflow.keras.optimizers import Adam

# Compile the model with a different learning rate
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Train the model with a different batch size and number of epochs
r = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=64
)

# Evaluate the model's performance on the test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.4458 - loss: 223.0134 - val_accuracy: 0.6494 - val_loss: 98.0794
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5969 - loss: 69.7251 - val_accuracy: 0.6364 - val_loss: 24.0263
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6269 - loss: 42.0538 - val_accuracy: 0.6494 - val_loss: 21.1987
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6256 - loss: 26.5444 - val_accuracy: 0.6883 - val_loss: 21.1037
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6783 - loss: 15.0380 - val_accuracy: 0.6364 - val_loss: 14.8262
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6930 - loss: 13.7979 - val_accuracy: 0.7143 - val_loss: 13.7047
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6716 - loss: 12.4026 - val_accuracy: 0.6688 - val_loss: 10.4966
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6788 - loss: 9.4532 - v

TypeError: Model.save() missing 1 required positional argument: 'filepath'

## Regularization

### Subtask:
Implement techniques like L1 or L2 regularization or dropout to prevent overfitting.


**Reasoning**:
Implement dropout layers in the model to prevent overfitting and then compile and train the model.



In [73]:
from tensorflow.keras.layers import Dropout

# Define a new Sequential model with dropout layers
model = Sequential()

# Add Dense layers with dropout
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5)) # Add dropout with a rate of 0.5
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3)) # Add dropout with a rate of 0.3
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2)) # Add dropout with a rate of 0.2

# Add the output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# Train the regularized model
r_regularized = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=64
)

# Evaluate the performance of the regularized model
loss_regularized, accuracy_regularized = model.evaluate(X_test, y_test, verbose=0)
print(f"Regularized Model Test Loss: {loss_regularized:.4f}")
print(f"Regularized Model Test Accuracy: {accuracy_regularized:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_106 (Dense)               │ (None, 128)            │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_108 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_109 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,521 (45.00 KB)

 Trainable params: 11,521 (45.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.4181 - loss: 12.2611 - val_accuracy: 0.6429 - val_loss: 3.7878
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6217 - loss: 4.5255 - val_accuracy: 0.6429 - val_loss: 1.6585
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6123 - loss: 2.9328 - val_accuracy: 0.6623 - val_loss: 0.6330
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5001 - loss: 2.8655 - val_accuracy: 0.6494 - val_loss: 0.6634
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5628 - loss: 2.2905 - val_accuracy: 0.6623 - val_loss: 0.9195
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5380 - loss: 1.8062 - val_accuracy: 0.6623 - val_loss: 0.8057
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6047 - loss: 1.8484 - val_accuracy: 0.6623 - val_loss: 0.7433
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5629 - loss: 1.4892 - val_accuracy: 0

**Reasoning**:
Initialize StratifiedKFold and lists to store cross-validation scores.



In [74]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5  # Number of folds
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
cvscores = []

**Reasoning**:
Loop through each fold, split the data, define, compile, fit, and evaluate the model for each fold, then store the scores.



In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Assuming X_scaled and y are already available from previous steps
for train_index, test_index in skf.split(X_scaled, y):
    X_train_fold, X_test_fold = X_scaled[train_index], X_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Define the model architecture within the loop
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train_fold.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Fit the model
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=64, verbose=0)

    # Evaluate the model
    scores = model.evaluate(X_test_fold, y_test_fold, verbose=0)
    print(f"Accuracy for fold: {scores[1]*100:.2f}%")
    cvscores.append(scores[1] * 100)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy for fold: 66.88%


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy for fold: 72.08%


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy for fold: 72.08%


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy for fold: 75.82%


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy for fold: 69.93%


**Reasoning**:
Calculate and print the mean and standard deviation of the cross-validation accuracies.



In [ ]:
print(f"Mean Accuracy: {np.mean(cvscores):.2f}%")
print(f"Standard Deviation of Accuracy: {np.std(cvscores):.2f}%")

## Ensemble methods

### Subtask:
Explore using ensemble methods like Random Forests or Gradient Boosting.


**Reasoning**:
Import the necessary libraries for Random Forest and evaluation metrics, then instantiate, train, predict, and evaluate the Random Forest model using the previously scaled training and test data.



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Instantiate a RandomForestClassifier model
# Using default parameters initially
rf_model = RandomForestClassifier(random_state=42)

# Train the Random Forest model on the scaled training data
# Assuming X_scaled is available from previous steps and matches the original data order before train_test_split
# We will use X_train and y_train from the train_test_split performed earlier
rf_model.fit(X_train, y_train)

# Predict on the scaled test data
y_pred_rf = rf_model.predict(X_test)

# Evaluate the performance of the Random Forest model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

# Print the evaluation metrics
print("Random Forest Model Evaluation:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-score: {f1_rf:.4f}")

## Algorithm selection

### Subtask:
Try different classification algorithms to see if another model performs better on your dataset.


In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

**Reasoning**:
Create instances of Logistic Regression, SVC, and K-Nearest Neighbors classifiers, train them on the training data, make predictions on the test data, and calculate the accuracy for each model.



In [81]:
# Create instances of the classifiers
lr_model = LogisticRegression(random_state=42)
svc_model = SVC(kernel='linear', random_state=42)

# Train the models
lr_model.fit(X_train, y_train)
svc_model.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test)
y_pred_svc = svc_model.predict(X_test)

# Evaluate accuracy
accuracy_lr = accuracy_score(y_test, y_pred_lr)
accuracy_svc = accuracy_score(y_test, y_pred_svc)


# Print the accuracy scores
print(f"Logistic Regression Accuracy: {accuracy_lr:.4f}")
print(f"SVC (Linear Kernel) Accuracy: {accuracy_svc:.4f}")
Model.save(lr_model)
Model.save(svc_model)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.7403
SVC (Linear Kernel) Accuracy: 0.7532


TypeError: Model.save() missing 1 required positional argument: 'filepath'

In [80]:
# 1. Define a list of the 8 feature names and their hint ranges
feature_names_with_hints = [
    ('Pregnancies', (0, 17)), # Changed hint to a tuple of min/max values
    ('Glucose', (44, 199)),
    ('BloodPressure', (24, 122)),
    ('SkinThickness', (7, 99)),
    ('Insulin', (14, 846)),
    ('BMI', (18.2, 67.1)),
    ('DiabetesPedigreeFunction', (0.078, 2.42)),
    ('Age', (21, 81))
    ]

# 2. Initialize an empty list to store the user's input values
user_input_values = []

# 3. Loop through the list of feature names and prompt for input with hints
for feature_name, hint_range in feature_names_with_hints:
    min_val, max_val = hint_range
    while True:
        try:
            value_str = input(f"Enter value for {feature_name} ({min_val}-{max_val}): ")
            # 4. Convert the user's input to a numerical type (float)
            value = float(value_str)

            # Add range validation
            if value < min_val or value > max_val:
                print(f"Input out of range. Please enter a value between {min_val} and {max_val}.")
            else:
                user_input_values.append(value)
                break # Exit the while loop if input is valid and within range
        except ValueError:
            print("Invalid input. Please enter a numerical value.")

print("\nUser input collected:", user_input_values)

KeyboardInterrupt: Interrupted by user

In [ ]:
# 1. Convert user input list to a NumPy array and reshape
user_input_array = np.array(user_input_values).reshape(1, -1)

# 2. Identify columns with potential outliers as zero
# These indices correspond to the original diabetes_df columns (0-indexed)
cols_with_outliers_as_zero_indices = [1, 2, 3, 4, 5] # Glucose, BloodPressure, SkinThickness, Insulin, BMI

# 3. Replace 0 values in the user input array for these columns with NaN
for col_index in cols_with_outliers_as_zero_indices:
    if user_input_array[0, col_index] == 0:
        user_input_array[0, col_index] = np.nan

# 4. Impute missing values (NaNs) with the mean of each column from the training data
# Need to access the means of the original training data (X_train) for imputation
# Calculate means from X_train (assuming X_train is available from previous steps)
training_means = X_train.mean()

for col_index in cols_with_outliers_as_zero_indices:
    if np.isnan(user_input_array[0, col_index]):
        user_input_array[0, col_index] = training_means[col_index]

# 5. Use the previously fitted scaler to transform the preprocessed user input array
# Assuming 'scaler' object is available from the preprocessing step
user_input_scaled = scaler.transform(user_input_array)

print("\nPreprocessed and scaled user input:", user_input_scaled)

**Reasoning**:
The preprocessed and scaled user input is ready for prediction. Use the loaded model to predict on the scaled input and display the prediction result.



In [ ]:
# Assuming 'model' is the trained model object (either the Keras model or the best performing sklearn model)

# For the best performing sklearn model (SVC)
# Assuming svc_model is the trained SVC model from the previous step
prediction = svc_model.predict(user_input_scaled)

# Define a function to display the prediction and suggestions
def suggestions(prediction, user_input_values, feature_names):
    """Displays the diabetes prediction and provides suggestions based on input values.

    Args:
        prediction: The prediction from the model (0 or 1).
        user_input_values: A list of the original user input values.
        feature_names: A list of the feature names.
    """
    if prediction[0] == 1:
        print("\nPrediction: Based on the provided information, the person is likely diabetic.")
        print("It is highly recommended to consult a doctor for a proper diagnosis and personalized treatment plan.")
        print("\nHere are some general suggestions based on your input values:")

        # Access the original user input values for targeted suggestions
        # Using indices as before, assuming the order is consistent

        # Suggestion for high Glucose (index 1)
        if user_input_values[1] > 120: # Using a threshold as an example
            print(f"- Your {feature_names[1][0]} level is high. Consider reducing sugar intake and refined carbohydrates.")

        # Suggestion for high BloodPressure (index 2)
        if user_input_values[2] > 80: # Using a threshold as an example
            print(f"- Your {feature_names[2][0]} is high. Focus on a low-sodium diet and regular physical activity.")

        # Suggestion for high BMI (index 5)
        if user_input_values[5] > 25: # Using a threshold as an example
            print(f"- Your {feature_names[5][0]} is in the higher range. Aim for weight management through a balanced diet and exercise.")

        # Suggestion for low BMI (though less common for diabetes, can still provide general health advice)
        if user_input_values[5] < 18.5: # Using a threshold as an example
             print(f"- Your {feature_names[5][0]} is in the lower range. Ensure you are consuming nutritious food to maintain a healthy weight.")


        # General suggestion for exercise
        print("- Incorporate regular physical activity into your routine.")
        print("- Maintain a healthy and balanced diet.")

    else:
        print("\nPrediction: Based on the provided information, the person is likely not diabetic.")
        print("Suggestion: Maintaining a healthy lifestyle with balanced diet and regular exercise is always beneficial for preventing diabetes.")

# Call the function to display the prediction and suggestions
# Assuming feature_names_with_hints is available from the input collection step
suggestions(prediction, user_input_values, feature_names_with_hints)